In [1]:
%load_ext autoreload
%autoreload now

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-08137aa2-e69b-5e74-8390-7997329b1336"

# Download and convert data

In [3]:
from document_segmentation.pagexml.annotations.renate_analysis import RenateAnalysis
from document_segmentation.settings import RENATE_ANALYSIS_DIR

N = None


sheet = RenateAnalysis()


sheet.download(RENATE_ANALYSIS_DIR, N)

Writing documents: 0doc [00:00, ?doc/s]


In [4]:
from document_segmentation.pagexml.annotations.renate_analysis import RenateAnalysisInv
from document_segmentation.settings import RENATE_ANALYSIS_DIR, RENATE_ANALYSIS_SHEETS

N = None


sheet = RenateAnalysisInv(RENATE_ANALYSIS_SHEETS[0])  # TODO: use both sheets
sheet.download(RENATE_ANALYSIS_DIR, N)

Writing documents:   0%|          | 0/26 [00:00<?, ?doc/s]

Writing documents: 100%|██████████| 26/26 [00:10<00:00,  2.39doc/s]


# Load Data

In [5]:
%autoreload now

In [6]:
TRAINING_DATA = 0.8

In [7]:
from document_segmentation.model.dataset import DocumentDataset

dataset: DocumentDataset = DocumentDataset.from_dir(RENATE_ANALYSIS_DIR)
dataset.shuffle()

len(dataset)

Reading JSON files: 100%|██████████| 104/104 [00:01<00:00, 92.40file/s] 


104

In [8]:
dataset._class_counts()

Counter({<Label.IN: 1>: 1907,
         <Label.BEGIN: 0>: 104,
         <Label.END: 2>: 100,
         <Label.OUT: 3>: 73})

In [9]:
dataset.class_weights()

[0.9904761904761905,
 0.05450733752620545,
 1.0297029702970297,
 1.4054054054054055]

In [10]:
training_data, test_data = dataset.split(TRAINING_DATA)

In [11]:
training_data._class_counts()

Counter({<Label.IN: 1>: 1569,
         <Label.BEGIN: 0>: 83,
         <Label.END: 2>: 79,
         <Label.OUT: 3>: 68})

In [12]:
test_data._class_counts()

Counter({<Label.IN: 1>: 338,
         <Label.BEGIN: 0>: 21,
         <Label.END: 2>: 21,
         <Label.OUT: 3>: 5})

# Train Model

In [13]:
BATCH_SIZE = 64
EPOCHS = 10
WEIGHTS = dataset.class_weights()

In [14]:
from document_segmentation.model.page_sequence_tagger import PageSequenceTagger

tagger = PageSequenceTagger()

In [15]:
tagger._device

'mps'

In [16]:
tagger

PageSequenceTagger(
  (_page_embedding): PageEmbedding(
    (_region_model): RegionEmbeddingSentenceTransformer(
      (_transformer_model): SentenceTransformer(
        (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: RobertaModel 
        (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
      )
      (_region_type): Embedding(9, 16)
      (_linear): Linear(in_features=784, out_features=512, bias=True)
    )
    (_rnn): LSTM(512, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
    (_linear): Linear(in_features=512, out_features=256, bias=True)
  )
  (_rnn): LSTM(256, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (_linear): Linear(in_features=512, out_features=4, bias=True)
  (_soft

In [22]:
tagger.train_(training_data, EPOCHS, BATCH_SIZE, WEIGHTS)

Training: 100%|██████████| 94/94 [00:05<00:00, 18.10batch/s]


[Loss:	0.243]


Training: 100%|██████████| 94/94 [00:05<00:00, 17.88batch/s]


[Loss:	0.243]


Training: 100%|██████████| 94/94 [00:05<00:00, 18.02batch/s]


...

[Loss:	0.243]


Training: 100%|██████████| 94/94 [00:05<00:00, 18.34batch/s]


[Loss:	0.243]


Training: 100%|██████████| 94/94 [00:05<00:00, 18.31batch/s]

[Loss:	0.243]


In [23]:
import torch

with open("page_sequence_tagger.pt", "xb") as f:
    torch.save(tagger, f)

# Evaluate Model

In [24]:
import sys

precision, recall, f1, accuracy = tagger.eval_(test_data, BATCH_SIZE, sys.stdout)

Predicted	Actual	Page ID	Text	Scores


Evaluating:  18%|█▊        | 4/22 [00:00<00:00, 38.44batch/s]

OUT	OUT	NL-HaNA_1.04.02_1547_0097.jpg		[0.00021971484238747507, 0.0001265001919819042, 9.484372753831849e-08, 0.9996535778045654]
OUT	OUT	NL-HaNA_1.04.02_1547_0098.jpg		[0.00043388621998019516, 0.0004980390076525509, 1.780613416713095e-07, 0.9990678429603577]
BEGIN	BEGIN	NL-HaNA_1.04.02_1547_0099.jpg	uwe; Hoog Edele agtb: gebiedende Heere en Heeren; 	[0.9530397057533264, 0.012777706608176231, 5.952011179033434e-06, 0.03417663648724556]
IN	IN	NL-HaNA_1.04.02_1547_0100.jpg	heen mne  dn d n n lel een w en de e sou co  an d;	[0.0020858743228018284, 0.9964151382446289, 0.0004819313180632889, 0.001016985042952001]
IN	IN	NL-HaNA_1.04.02_1547_0101.jpg	te Nambedle, den prince Cartadavile, en Moerianato	[0.0010368991643190384, 0.9930505752563477, 0.005753073841333389, 0.00015941268065944314]
IN	IN	NL-HaNA_1.04.02_1547_0102.jpg	werden; war op de ehien anmandijn lae telkens tot 	[0.0007002142956480384, 0.9838862419128418, 0.015348009765148163, 6.552737613674253e-05]
IN	IN	NL-HaNA_1.04.02_1547_0103

Evaluating:  36%|███▋      | 8/22 [00:00<00:00, 22.70batch/s]

BEGIN	BEGIN	NL-HaNA_1.04.02_1490_0583.jpg	Copije Secrete Resolutien; genomen bij de Ho: Rege	[0.9998180270195007, 0.0001491154107498005, 9.984536575302627e-08, 3.276505594840273e-05]
IN	IN	NL-HaNA_1.04.02_1490_0584.jpg	in presentie; van zijn Ed. t te; volbrengen, soo s	[0.004640901926904917, 0.9898576736450195, 0.005366253666579723, 0.00013527320697903633]
IN	IN	NL-HaNA_1.04.02_1490_0585.jpg	dat 's Comp. s dienaren en wel voorna„; mentlijk p	[0.001961312023922801, 0.9759576916694641, 0.022016899660229683, 6.41572114545852e-05]
IN	IN	NL-HaNA_1.04.02_1490_0586.jpg	bij ons te boeck staen voor niet wel; geintentione	[0.001705485163256526, 0.9658207297325134, 0.03242245316505432, 5.1397470087977126e-05]
IN	IN	NL-HaNA_1.04.02_1490_0587.jpg	goetgevonden dat dese ontbiedinge; bij ons gemeen 	[0.00170176534447819, 0.9612826704978943, 0.03696996718645096, 4.563002585200593e-05]
IN	IN	NL-HaNA_1.04.02_1490_0588.jpg	schaap harder herwaerts aen te doen; overkomen, ge	[0.0017693917034193873, 0.959929

Evaluating:  82%|████████▏ | 18/22 [00:00<00:00, 33.30batch/s]

BEGIN	BEGIN	NL-HaNA_1.04.02_2356_0028.jpg	Mocha den 15 augustus 1734:; Van; Mocha onder Dato	[0.9999734163284302, 2.1198251488385722e-05, 7.084087627617919e-08, 5.3206749726086855e-06]
IN	IN	NL-HaNA_1.04.02_2356_0029.jpg	Van Bovia onder dato 15 augustus 1734; Van; Colba 	[0.0002637459256220609, 0.999639630317688, 7.285345054697245e-05, 2.3691380192758515e-05]
IN	IN	NL-HaNA_1.04.02_2356_0030.jpg	Sacha Onder Dato 15. Aug: 1734; Van; Cocha onder D	[0.00010390162788098678, 0.9998053908348083, 7.916913455119357e-05, 1.1566015018615872e-05]
IN	IN	NL-HaNA_1.04.02_2356_0031.jpg	Van Mocha onder dato 15 aug:s 1734; Van; Mocha ond	[9.593649883754551e-05, 0.9998134970664978, 7.921530777821317e-05, 1.1348437510605436e-05]
IN	IN	NL-HaNA_1.04.02_2356_0032.jpg	Cocha onder dato 18 Augutus 1744; Van; Augustus; V	[0.00010297694097971544, 0.9998037219047546, 8.166378393070772e-05, 1.1629648724920116e-05]
IN	IN	NL-HaNA_1.04.02_2356_0033.jpg	Van Norsa onder dato 15 Aug 1734:; Van Mocha onder	[9.888029308058

Evaluating: 100%|██████████| 22/22 [00:00<00:00, 30.76batch/s]

BEGIN	BEGIN	NL-HaNA_1.04.02_3248_0877.jpg	Op Huijden den 5: October a„o 1761: voor mij Wolfe	[0.9999746084213257, 2.06444165087305e-05, 7.695143011687833e-08, 4.609969437296968e-06]
IN	IN	NL-HaNA_1.04.02_3248_0878.jpg	op die wijze gedurende hun verblijf aldaar, en ond	[0.0002955368545372039, 0.9996078610420227, 7.158010703278705e-05, 2.499421862012241e-05]
IN	IN	NL-HaNA_1.04.02_3248_0879.jpg	niet krijgende ! de Xullas aandoen, en overweldige	[0.00010563666728558019, 0.9998038411140442, 7.878089672885835e-05, 1.1790100870712195e-05]
IN	IN	NL-HaNA_1.04.02_3248_0880.jpg	van Christoffel dias, en Adriaan Rijkschroef clerc	[8.880866516847163e-05, 0.9998216032981873, 7.93698345660232e-05, 1.0287281838827766e-05]
IN	IN	NL-HaNA_1.04.02_3248_0881.jpg	Heden den 8:e Maart 1762: Compareerde voor mij Ioh	[8.860236994223669e-05, 0.999821126461029, 7.981384260347113e-05, 1.0543864846113138e-05]
IN	IN	NL-HaNA_1.04.02_3248_0882.jpg	dog Een van derzelver prauwen bleeft op de droogte	[9.352608321933076e-0

In [25]:
import csv

from document_segmentation.pagexml.datamodel.label import Label

writer = csv.DictWriter(
    sys.stdout, fieldnames=["Metric"] + [label.name for label in Label], delimiter="\t"
)

writer.writeheader()

for metric in precision, recall, f1:
    assert metric.average is None

    scores: list[float] = metric.compute().tolist()
    writer.writerow(
        {"Metric": metric.__class__.__name__}
        | {label.name: f"{score:.4f}" for label, score in zip(Label, scores)}
    )

assert accuracy.average is not None

print(
    f"{metric.__class__.__name__} ({accuracy.average} average):\t{accuracy.compute().item():.4f}",
    file=sys.stdout,
)

Metric	BEGIN	IN	END	OUT
MulticlassPrecision	0.9091	1.0000	0.9524	0.8333
MulticlassRecall	0.9524	0.9941	0.9524	1.0000
MulticlassF1Score	0.9302	0.9970	0.9524	0.9091
MulticlassF1Score (micro average):	0.9896
